In [925]:
import numpy as np
import datetime
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.mixture import GMM
import xgboost
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score
from scipy.sparse import hstack
from scipy.sparse import csc_matrix
from collections import Counter
from sklearn.cross_validation import KFold
from sklearn.cross_validation import train_test_split
pd.set_option('display.max_columns', 80) 
pd.set_option('display.max_rows', 1000) 
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

1) Можно узнать долю каждого класса отправив одни единицы через полученный f1score.

2) Зная долю каждого класса и используя долю классов в сабмите, получая скор (а также понимаю сколько в сумме объектов) можно восстановить confusion matrix, так как получается 4 уравнения с 4-мя неизвестными: TP, TN, FP, FN.

3) Строим дерево (мне в процессе было удобно делать каскад): делаем сплит, смотрим confusion matrix и понимаем сколько объектов в листе. Обучаемся на лидерборде. Для разбиение вершины можно попробовать несколько разных сплитов и просто выбрать лучший. Вызов в том, что нужно сразу понимать какой сплит будет хорошим, так как количество сабмитов в день ограниченно. 

Далее приведен уже финальный код где обозначаем за единицу, если удовлетворяет 4-м условиям.

In [926]:
data = pd.read_csv("../data/avito/log.csv")

In [927]:
def get_confusion_matrx(f1_score, one_share):
    x, y, z = np.linalg.solve([[1, 0, 1], [1, 1, 0], [2 - 2/f1_score, 1, 1]], [0.10503025, one_share, 0])
    print(np.array([x*100, z*100]))
    print(np.array([y*100, (1 - x - y - z)*100]))

In [928]:
def get_time(x):
    return datetime.datetime.strptime(x,  "%Y-%m-%d %H:%M:%S")

time = data.event_date.apply(get_time)
cook_num = data.groupby(data.cookie_id).apply(len)
tim2 = (time - min(time)).apply(lambda x: x.total_seconds())

def get_time2(x):
    x = np.array(x)
    if len(x) == 1:
        return -1
    return min(x[1:] - x[:-1])
min_time = tim2.groupby(data.cookie_id).apply(get_time2) #минимальное время в секунах между действиями 

In [ ]:
windows = data.useragent.fillna('na').apply(lambda x: 'Windows' in x)
is_windows = windows.groupby(data.cookie_id).max() #есть ли 'Windows' хотя бы в одном из useragent для данного cookie_id

In [957]:
user_id = data.user_id.notnull().groupby(data.cookie_id).mean()

In [931]:
avito = data.useragent.fillna('na').apply(lambda x: 'AVITO' in x)
is_avito = avito.groupby(data.cookie_id).min() #Во всех useragent для данного cookie_id есть 'AVITO'

In [933]:
# Человек, если более 1 лога на cookie_id, не винда, useragent со словом авито
# и минимальное время между действимя менее 30 секунд
index = (cook_num > 1) & (1 - is_windows) & (min_time < 30) & is_avito

Процесс улучшения:

0.84 - (cook_num > 1)

0.88 - (cook_num > 1) & (1 - is_windows)

0.94 - (cook_num > 1) & (1 - is_windows) & (min_time < 30)

0.95 - (cook_num > 1) & (1 - is_windows) & (min_time < 30) & is_avito

Обязательно нужно куда-нибудь впихнуть хгбуст.

Можно для каждого cookie_id сделать признаковое описание без использование предыдущих фичей и взять уже готовые метки. Пуститься в кросс-валидацию и напредсказывать новых меток. Скор улучшился до 0.96.

In [937]:
X = data.eventtype_id.groupby(data.cookie_id).apply(lambda x: len(x.unique())/len(x))
X = pd.DataFrame(X, columns=['eventtype_id_share'])
for item in data.eventype_name.unique():
    X['eventype_name_' + item] = (data.eventype_name == item).groupby(data.cookie_id).mean()
X['cook_num'] = cook_num
X['Android'] = (data.ClientSideAppName == 'Android App').groupby(data.cookie_id).mean()
X['is_event_pageview_share'] = data.is_event_pageview.groupby(data.cookie_id).mean()
for item in data.eventtype_group.unique():
     X['eventtype_group_' + item] = (data.eventtype_group == item).groupby(data.cookie_id).mean()
X['user_id'] = user_id
time2 = time.apply(lambda x: x.minute + x.hour * 60)
tmp = time2.groupby(data.cookie_id).apply(lambda x: np.array(x))
time_night_share = tmp.apply(lambda x: ((x > 100) & (x < 600)).mean())
X['time_night_share'] = time_night_share

In [1002]:
y = (cook_num > 1) & (1 - is_windows) & (min_time < 30) & is_avito

In [1005]:
model = xgboost.XGBClassifier(n_estimators=59, 
                              max_depth=6, 
                              seed=100)

In [1026]:
cv = KFold(len(X), n_folds=5, shuffle=True, random_state=241)
p = pd.Series(index = X.index)
for train, test in cv:
    model.fit(X.iloc[train], y.iloc[train])
    p.iloc[test] = model.predict_proba(X.iloc[test])[:, 1]

In [1085]:
sub = (p > 0.5).apply(int).reset_index()
sub.columns = ['cookie_id', 'IsHuman']
sub.to_csv('B.csv', index=False, sep=',')

Итоговый confusion matrix в процентах от выборки:

In [1087]:
get_confusion_matrx(0.9621, sub.IsHuman.mean())

[ 10.42109455   0.08193045]
[  0.73910579  88.75786921]
